In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from copy import deepcopy
from ax import optimize
from ax.plot.contour import plot_contour
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.plot.trace import optimization_trace_single_method

import json
import torch
mps_device = torch.device("mps")

## Simple Linear Dataset

Let's start with a ground-truth linear model that we know a linear model should be able to recover.

In [2]:
# generate a dataset
NROW = 1000
NCOL = 5

random.seed(1010)

data = np.random.rand(NROW, NCOL)
X = pd.DataFrame(data, columns=[f"col_{i}" for i in range(1, NCOL+1)])
y = X["col_1"] + 2 * X["col_2"] + 3 * X["col_3"] + 4 * X["col_4"] + 5 * X["col_5"] + np.random.uniform(0, 0.5, NROW)

In [3]:
# train-test split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1010)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

#### Optimize/Train the Model on the Training Set (and report optimal hyperparams)

In [4]:
# code adapted from Mohammed's q2 optimization notebook
model_dict = {"RandomForestRegressor":{"model":RandomForestRegressor,
					"params":[{"name":"n_estimators", "type":"range", "bounds":[2,200]},
							{"name":"max_depth", "type":"range", "bounds":[1,5]},
							{"name":"min_samples_leaf", "type":"range", "bounds":[1,5]}]},
                "MLPRegressor":{"model": MLPRegressor,
					"params":[{"name":"activation", "type":"choice", "values":["identity", "logistic", "tanh", "relu"]},
							{"name":"solver", "type":"choice", "values":["sgd", "adam"]},
                            {"name": "alpha", "type": "range", "bounds":[0.0001,1.0]},
                            {"name":"learning_rate", "type":"choice", "values":["constant", "adaptive"]},
                            {"name":"learning_rate_init", "type":"range", "bounds":[0.0001, 0.01]},
                            {"name":"max_iter", "type":"range", "bounds":[200, 500]}
                        ]},
			   "ElasticNet":{ "model": ElasticNet,
					"params":[{"name": "alpha", "type": "range", "bounds":[0.001,1.0]},
							{"name": "l1_ratio", "type": "range", "bounds":[0.0,1.0]},
							{"name": "max_iter", "type": "range", "bounds":[200, 2000]},
							{"name": "selection", "type": "choice", "values":["cyclic", "random"]}
						]}
			}

def q2_baseline_models(estimator, X, y):
    baseline_models = []
    for loo_index in X.index:
        if hasattr(estimator, "random_state"):
            estimator.random_state = loo_index	
        baseline_models.append(deepcopy(estimator.fit(X=X.drop(loo_index).values, y=y.drop(loo_index).values)))
        
    return baseline_models

# This function computes q^2 (used as evaluation for the models)
def q2_score(estimator, X, y):
    models = q2_baseline_models(estimator, X, y)
    q2_means = []
    q2_preds = []
    
    for loo_index in X.index:
        q2_means.append(y.drop(loo_index).mean())
        q2_preds.append(models[loo_index].predict(np.array(X.iloc[loo_index]).reshape(1,-1))[0])

    q2 = 1 - np.sum((np.array(q2_preds) - np.array(y).flatten())**2) / np.sum((np.array(q2_means) - np.array(y))**2)
    
    return q2

def get_optimal_model(X, y, model_type, total_trials):
    best_parameters, best_values, experiment, model = optimize(
        parameters= model_dict[model_type]["params"],
        evaluation_function=lambda p: q2_score(model_dict[model_type]["model"](**p), X, y),
        minimize=False,
        total_trials=total_trials,
    )
    optimal_parameters = best_parameters 
    optimization_q2 = best_values[0]["objective"]

    return(optimal_parameters, optimization_q2)

In [5]:
# print the training q^2 on the training dataset
optimal_params, optimal_q2 = get_optimal_model(X_train, y_train, model_type = "ElasticNet", total_trials = 10)

[INFO 03-27 01:00:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter alpha. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter l1_ratio. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:16] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter max_iter. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:16] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter selection. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in par

In [6]:
optimal_q2

0.9953944024104368

In [7]:
optimal_params

{'alpha': 0.001,
 'l1_ratio': 0.3864746851088902,
 'max_iter': 1584,
 'selection': 'cyclic'}

#### Evaluate the model on the true held-out data

In [8]:
# print the evaluation q^2 on the evaluation dataset
model_optimal = ElasticNet(alpha = optimal_params["alpha"],
                           l1_ratio= optimal_params["l1_ratio"],
                           max_iter = optimal_params["max_iter"],
                           selection = optimal_params["selection"])

No surprise --- we found the ground truth coefficients, so it works out-of-sample.

In [9]:
q2_score(model_optimal, X_test, y_test)

0.995117341885527

## Complex Dataset with Omitted Variables and Nonlinearities

Let's create a dataset with nonlinear relationships, omitted variables, and lots of noise --- making it more likely that the original model will overfit.

In [10]:
# generate a dataset
NROW = 1000
NCOL = 5

random.seed(1010)

data = np.random.rand(NROW, NCOL)
X = pd.DataFrame(data, columns=[f"col_{i}" for i in range(1, NCOL+1)])
y = X["col_1"] + 2 * X["col_2"] + 3 * X["col_3"]**3 + 4 * X["col_4"] * X["col_5"] + np.random.uniform(0, 2, NROW)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1010)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# drop cols 1 and 5 so that we have omitted variables
X_train = X_train.drop(["col_1", "col_5"], axis = 1)
X_test = X_test.drop(["col_1", "col_5"], axis = 1)

### Model 1: ElasticNet

In [11]:
optimal_params, optimal_q2 = get_optimal_model(X_train, y_train, model_type = "ElasticNet", total_trials = 30)

[INFO 03-27 01:00:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter alpha. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter l1_ratio. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:21] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter max_iter. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:21] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter selection. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in par

In [12]:
optimal_q2

0.5795978511781202

In [13]:
optimal_params

{'alpha': 0.001,
 'l1_ratio': 0.5401704336034272,
 'max_iter': 1306,
 'selection': 'cyclic'}

In [14]:
# print the evaluation q^2 on the evaluation dataset
model_optimal = ElasticNet(alpha = optimal_params["alpha"],
                           l1_ratio= optimal_params["l1_ratio"],
                           max_iter = optimal_params["max_iter"],
                           selection = optimal_params["selection"])

The overall Q^2 is now much lower, but we're not seeing much of a bias.

In [15]:
q2_score(model_optimal, X_test, y_test)

0.5389667306371033

### Model 2: Random Forest

In [16]:
# try with a random forest model
optimal_params, optimal_q2 = get_optimal_model(X_train, y_train, model_type = "RandomForestRegressor", total_trials = 10)

[INFO 03-27 01:00:47] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter n_estimators. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:47] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter max_depth. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:47] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter min_samples_leaf. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-27 01:00:47] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='n_estimators', parameter_type=INT, range=[2, 200]), RangeParameter(name='max_depth', parameter_type=INT, range

In [17]:
optimal_q2

0.6058759202486838

In [18]:
optimal_params

{'n_estimators': 82, 'max_depth': 5, 'min_samples_leaf': 3}

In [19]:
model_optimal = RandomForestRegressor(n_estimators = optimal_params["n_estimators"],
                           max_depth= optimal_params["max_depth"],
                           min_samples_leaf = optimal_params["min_samples_leaf"])

In [20]:
q2_score(model_optimal, X_test, y_test)

0.5394947091992544

### Model 3: Vanilla Neural Network (Multi-Layered Perceptron)

In [21]:
# let's go even fancier and try a vanilla NN
optimal_params, optimal_q2 = get_optimal_model(X_train, y_train, model_type = "MLPRegressor", total_trials = 10)

[INFO 03-27 01:07:49] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter activation. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/ax/core/parameter.py:518: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "activation". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.

/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/ax/core/parameter.py:518: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "activation". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 03-27 01:07:49] ax.service.util

In [22]:
optimal_q2

0.6208153315219198

In [23]:
optimal_params

{'alpha': 0.21560872699581085,
 'learning_rate_init': 0.009074210975226015,
 'max_iter': 447,
 'activation': 'relu',
 'solver': 'adam',
 'learning_rate': 'constant'}

In [24]:
model_optimal = MLPRegressor(alpha = optimal_params["alpha"],
                           learning_rate_init= optimal_params["learning_rate_init"],
                           max_iter= optimal_params["max_iter"],
                           activation = optimal_params["activation"],
                           solver = optimal_params["solver"],
                           learning_rate = optimal_params["learning_rate"])

In [25]:
q2_score(model_optimal, X_test, y_test)

0.5823506962780255